In [1]:
import geneformer
print(geneformer.__file__)
import os
GPU_NUMBER = [2]
os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(s) for s in GPU_NUMBER])
os.environ["NCCL_DEBUG"] = "INFO"
print(geneformer.tokenizer.TOKEN_DICTIONARY_FILE)
from collections import Counter
import datetime
import pickle
import dill
import subprocess
import seaborn as sns; sns.set()
from datasets import load_from_disk
from sklearn.metrics import accuracy_score, f1_score
from transformers import BertForSequenceClassification
from transformers import Trainer
from transformers.training_args import TrainingArguments

from geneformer import DataCollatorForCellClassification
import scanpy as sc
from geneformer import TranscriptomeTokenizer
import numpy as np
import torch

/mnt/nfs/wbzhang/geneformer-master/Geneformer/myenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/mnt/nfs/wbzhang/geneformer-master/Geneformer/myenv/lib/python3.8/site-packages/geneformer/__init__.py
/mnt/nfs/wbzhang/geneformer-master/Geneformer/myenv/lib/python3.8/site-packages/geneformer/token_dictionary.pkl


In [2]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # calculate accuracy and macro f1 using sklearn's function
    acc = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average='macro')
    return {
      'accuracy': acc,
      'macro_f1': macro_f1
    }

In [3]:
folder_path = '/mnt/nfs/wbzhang/data/celltype_annotation/zheng68k_final/data/geneformer/5_fold_arrow/'
pretrain_model_path = '/mnt/nfs/xz/geneformer/Geneformer34m/examples/pretraining_new_model/output_own/models/230813_170132_geneformer_30M_L12_emb512_SL2048_E3_B12_LR0.0005_LSlinear_WU10000_Oadamw_DS64/models'
finetune_path = "/mnt/nfs/wbzhang/data/celltype_annotation/zheng68k_final/Model/geneformer/we_12L_finetune/"
fold_number = 5
for i in range(1):
    print(i)
    fold_path = folder_path + 'fold' + str(i) + '_test_arrow.dataset'
    test_dataset = load_from_disk(fold_path)
    total = test_dataset.rename_column("cell_type","label")
    total = total.remove_columns("organ")
    target_names = list(Counter(total["label"]).keys())
    target_name_id_dict = dict(zip(target_names,[i for i in range(len(target_names))]))

    def classes_to_ids(example):
        example["label"] = target_name_id_dict[example["label"]]
        return example
    labeled_total = total.map(classes_to_ids, num_proc=16)
    testset = labeled_total
    
    model = BertForSequenceClassification.from_pretrained(pretrain_model_path, 
                                                  num_labels=len(target_name_id_dict.keys()),
                                                  output_attentions = False,
                                                  output_hidden_states = False).to("cuda")
    finetune_dir = finetune_path + 'fold' + str(i) + '_model/pytorch_model.bin'
    print(finetune_dir)
    checkpoint = torch.load(finetune_dir)
    model.load_state_dict(checkpoint)

    trainer = Trainer(
    model=model,
    data_collator=DataCollatorForCellClassification(),
    compute_metrics=compute_metrics
    )
    predictions = trainer.predict(testset)
    print('The prediction result of '+str(i)+' fold:')
    print(compute_metrics(predictions))

0


Map (num_proc=16): 100%|███████████████████████████████████████████████████████████████████████████████| 13342/13342 [00:00<00:00, 52634.29 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /mnt/nfs/xz/geneformer/Geneformer34m/examples/pretraining_new_model/output_own/models/230813_170132_geneformer_30M_L12_emb512_SL2048_E3_B12_LR0.0005_LSlinear_WU10000_Oadamw_DS64/models and are newly initialized: ['bert.pooler.dense.bias', 'classifier.weight', 'bert.pooler.dense.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/mnt/nfs/wbzhang/data/celltype_annotation/zheng68k_final/Model/geneformer/we_12L_finetune/fold0_model/pytorch_model.bin


/mnt/nfs/wbzhang/geneformer-master/Geneformer/myenv/lib/python3.8/site-packages/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


The prediction result of 0 fold:
{'accuracy': 0.025183630640083946, 'macro_f1': 0.02372621801398035}


In [ ]:
model_state_dict

In [45]:
predictions.predictions

array([[ 0.2664529 , -1.0617732 , -0.97898096, ..., -2.1967382 ,
        -2.2737122 , -3.4333665 ],
       [-2.2471647 , -0.5895008 ,  0.4020873 , ..., -2.461241  ,
        -2.33209   , -4.020839  ],
       [ 1.5664858 , -1.843969  ,  0.6841839 , ..., -2.5160458 ,
        -2.4820912 , -3.8923113 ],
       ...,
       [-1.2024128 , -2.6249254 , -2.2684877 , ..., -2.0823631 ,
        -1.6634806 , -2.099236  ],
       [-2.5956922 ,  6.1489105 , -1.2428662 , ..., -1.2053187 ,
        -2.6050742 , -2.2316387 ],
       [-1.0306069 , -1.6278262 , -0.44036603, ..., -2.3900173 ,
        -2.445148  , -3.801106  ]], dtype=float32)

In [31]:
testset

Dataset({
    features: ['input_ids', 'label', 'length'],
    num_rows: 13342
})

In [32]:
import dill
folder_path = '/mnt/nfs/wbzhang/data/celltype_annotation/zheng68k'
with open(folder_path+ '/'+'test.pkl','rb') as f:
    test = dill.load(f)


Dataset({
    features: ['input_ids', 'label', 'length'],
    num_rows: 18358
})

In [9]:
folder_path = '/mnt/nfs/wbzhang/data/celltype_annotation/zheng68k_final/data/geneformer/5_fold_arrow/'
pretrain_model_path = '/mnt/nfs/xz/geneformer/Geneformer34m/examples/pretraining_new_model/output_own/models/230813_170132_geneformer_30M_L12_emb512_SL2048_E3_B12_LR0.0005_LSlinear_WU10000_Oadamw_DS64/models'
finetune_path = "/mnt/nfs/wbzhang/data/celltype_annotation/zheng68k_final/Model/geneformer/we_12L_finetune/"
fold_number = 5
for i in range(1):
    fold_path = folder_path + 'fold' + str(i) + '_test_arrow.dataset'
    test_dataset = load_from_disk(fold_path)
    total = test_dataset.rename_column("cell_type","label")
    total = total.remove_columns("organ")
    target_names = list(Counter(total["label"]).keys())
    target_name_id_dict = dict(zip(target_names,[i for i in range(len(target_names))]))
    
    model = BertForSequenceClassification.from_pretrained(pretrain_model_path, 
                                                  num_labels=len(target_name_id_dict.keys()),
                                                  output_attentions = False,
                                                  output_hidden_states = False).to("cuda")

    def classes_to_ids(example):
        example["label"] = target_name_id_dict[example["label"]]
        return example
    labeled_total = total.map(classes_to_ids, num_proc=16)
    testset = labeled_total
    '''
    finetune_dir = finetune_path + 'fold' + str(i) + '_model/pytorch_model.bin'
    checkpoint = torch.load(finetune_dir)
    model.load_state_dict(checkpoint)

    trainer = Trainer(
    model=model,
    data_collator=DataCollatorForCellClassification(),
    compute_metrics=compute_metrics
    )

    predictions = trainer.predict(testset)
    print('The prediction result of '+str(i)+' fold:')
    print(compute_metrics(predictions))
    '''

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /mnt/nfs/xz/geneformer/Geneformer34m/examples/pretraining_new_model/output_own/models/230813_170132_geneformer_30M_L12_emb512_SL2048_E3_B12_LR0.0005_LSlinear_WU10000_Oadamw_DS64/models and are newly initialized: ['bert.pooler.dense.weight', 'classifier.bias', 'bert.pooler.dense.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
testset

Dataset({
    features: ['input_ids', 'label', 'length'],
    num_rows: 13342
})